# Problema de localización de instalaciones

## Objetivo y Prerrequisitos
En este ejemplo, resolveremos un problema de ubicación de instalaciones donde queremos construir almacenes para abastecer a un cierto número de hospitales. Construiremos un modelo de programación entera mixta (MIP) de este problema, implementaremos este modelo en la interfaz de Python de PuLP + Cbc y calcularemos una solución óptima.



## Motivación

El estudio de los problemas de localización de instalaciones, también conocidos como análisis de ubicación [1], es una rama de la investigación de operaciones y la geometría computacional que se preocupa por la ubicación óptima de instalaciones para minimizar los costos ***e.g.*** de transporte, mientras se consideran factores como evitar colocar materiales peligrosos cerca de viviendas y la ubicación de las instalaciones de competidores.



Los problemas de ubicación de instalaciones tienen aplicaciones en una amplia variedad de sectores e industrias. Por ejemplo, en la gestión de la cadena de suministro y la logística, este problema se puede utilizar para encontrar la ubicación óptima de tiendas, fábricas, almacenes, etc. Otras aplicaciones van desde la política pública (por ejemplo, la ubicación de policías, ambulancias u hospitales en una ciudad), las telecomunicaciones (por ejemplo, torres de celulares en una red) e incluso la física de partículas (por ejemplo, la distancia de separación entre cargas repulsivas). Otra aplicación del problema de ubicación de instalaciones es determinar las ubicaciones para el equipo de transmisión de gas natural. Finalmente, los problemas de ubicación de instalaciones se pueden aplicar al análisis de clústeres.

## Descripción del problema

Una red de hospitales en Colombia necesita construir almacenes para suministrar insumos médicos a sus sedes en el norte del país. Las ubicaciones de los hospitales ya han sido decididas, pero aún no se ha determinado la ubicación de los almacenes.

Se han identificado varias ubicaciones potenciales para los almacenes, pero se deben tomar decisiones sobre cuántos almacenes abrir y en qué ubicaciones candidatas construirlos.

Abrir muchos almacenes sería ventajoso, ya que esto reduciría la distancia promedio que un camión tendría que recorrer desde el almacén hasta el hospital, y por lo tanto, reduciría el costo de entrega. Sin embargo, la apertura de un almacén tiene un costo fijo asociado.

En este ejemplo, nuestro objetivo es encontrar el equilibrio óptimo entre el costo de entrega y el costo de construir nuevas instalaciones.

## Solución planteada

La programación matemática es un enfoque declarativo en el que el modelador formula un modelo de optimización matemática que captura los aspectos clave de un problema de decisión complejo. El motoro de optimización, ---eg.--- Gurobi, CPLEX, SCIP, HIGHS, etx.,  resuelve tales modelos utilizando técnicas matemáticas-algrebáricas y algoritmos computacionales de última generación.

Un modelo de optimización matemática tiene cinco componentes, a saber:

* Conjuntos e índices.
* Parámetros.
* Variables de decisión.
* Función(es) objetivo.
* Restricciones.

Presentamos a continuación una formulación MIP para el problema de ubicación de instalaciones.


## Formulación del Modelo

### Conjuntos e Índices
$i \in I$: Índice y conjunto de ubicaciones de hospitales (o clientes).

$j \in J$: Índice y conjunto de ubicaciones candidatas de almacenes (o instalaciones).

### Parámetros
$f_{j} \in \mathbb{R}^+$: Costo fijo asociado con la construcción de la instalación $j \in J$.

$d_{i,j} \in \mathbb{R}^+$: Distancia entre la instalación $j \in J$ y el cliente $i \in I$.

$c_{i,j} \in \mathbb{R}^+$: Costo de envío entre el sitio candidato de la instalación $j \in J$ y la ubicación del cliente $i \in I$. Se asume que este costo es proporcional a la distancia entre la instalación y el cliente. Es decir, $c_{i,j} = \alpha \cdot d_{i,j}$, donde $\alpha$ es el costo por milla de conducción, ajustado para incorporar el número promedio de viajes que se esperaría que un camión de entrega realizara durante un período de cinco años.

### Variables de Decisión
$select_{j} \in {0, 1 }$: Esta variable es igual a 1 si construimos una instalación en la ubicación candidata $j \in J$; y 0 en caso contrario.

$0 \leq assign_{i,j} \leq 1$: Esta variable continua no negativa determina la fracción de suministro recibida por el cliente $i \in I$ de la instalación $j \in J$.


### Función Objetivo

Costos totales. Queremos minimizar el costo total de abrir y operar las instalaciones. Esta es la suma del costo de abrir las instalaciones y el costo relacionado con el envío entre las instalaciones y los clientes. Este costo total mide el compromiso entre el costo de construir una nueva instalación y el costo total de envío durante un período de cinco años.
\begin{equation}
\text{Max} \quad Z = \sum_{j \in J} f_{j} \cdot select_{j} + \sum_{j \in J} \sum_{i \in I} c_{i,j} \cdot assign_{i,j}
\tag{0}
\end{equation}

### Restricciones
#### Demanda.

 Para cada cliente $i \in I$, aseguramos que se cumpla su demanda. Es decir, la suma de la fracción recibida de cada instalación para cada cliente debe ser igual a 1:
\begin{equation}
\sum_{j \in J} assign_{i,j} = 1 \quad \forall i \in I
\tag{1}
\end{equation}

####  Envío. 

Debemos asegurarnos de que solo enviemos desde la instalación $j \in J$ si esa instalación realmente ha sido construida.
\begin{equation}
assign_{i,j} \leq select_{j} \quad \forall i \in I \quad \forall j \in J
\tag{2}
\end{equation}

## Implementación Python 

Este ejemplo considera dos hospitales y nueve almacenes candidatos. Las coordenadas de cada hospital se proporcionan en la siguiente tabla.

| <i></i> | Coordinates |  
| --- | --- | 
| Hospital 1 | (0,1.5) |
| Hospital 2 | (2.5,1.2) |

La siguiente tabla muestra las coordenadas de los sitios de almacén candidatos y el costo fijo de construir el almacén en millones de UDS.

| <i></i> | coordenadas | costo |
| --- | --- |  --- |
| Warehouse 1 | (0,0) | 3 |
| Warehouse 2 | (0,1) | 2 |
| Warehouse 3 | (0,2) | 3 |
| Warehouse 4 | (1,0) | 1 |
| Warehouse 5 | (1,1) | 3 | 
| Warehouse 6 | (1,2) | 3 |
| Warehouse 7 | (2,0) | 4 |
| Warehouse 8 | (2,1) | 3 |  
| Warehouse 9 | (2,2) | 2 |


El costo por Km es $\$1$ millón USD.

## Python Implementation

Ahora importamos el módulo Pulp Python y otras bibliotecas de Python que nos ayudarán en la construcción del modelo. Luego, inicializaremos las estructuras de datos con los datos dados.

In [46]:
setup_cost = [np.random.randint(1, 6) for i in range(20)]
setup_cost

[5, 5, 4, 3, 5, 1, 2, 2, 1, 5, 1, 3, 5, 3, 1, 2, 2, 2, 2, 2]

In [47]:
from itertools import product
from math import sqrt

import pulp as plp


# tested with Gurobi v9.0.0 and Python 3.7.0

# Parameters
customers = [(0,1.5), 
             (2.5,1.2), 
             (2.5,2.5), 
             (1.5,2.5), 
             (0.5,2.5), 
             (0,0.5), 
             (2.5,0.5), 
             (2.5,2), 
             (1.5,2), 
             (0.5,2), 
             (0,1), 
             (2.5,1), 
             (2.5,1.5), 
             (1.5,1.5), 
             (0.5,1.5), 
             (0,0), 
             (2.5,0), 
             (2.5,0.5), 
             (1.5,0.5), 
             (0.5,0.5)]
facilities = [(0.18, 1.02),
            (0.6, 2.85),
            (1.23, 0.84),
            (0.63, 1.5),
            (1.71, 1.74),
            (0.24, 0.03),
            (1.2, 2.28),
            (0.42, 2.13),
            (0.09, 2.49),
            (1.62, 2.64),
            (2.46, 0.39),
            (1.83, 0.09),
            (0.78, 1.68),
            (2.76, 0.87),
            (0.48, 1.86),
            (1.86, 0.63),
            (1.59, 2.7),
            (2.04, 1.44),
            (0.54, 1.89),
            (0.06, 2.94)]
setup_cost = [1, 4, 1, 4, 3, 4, 5, 3, 5, 1, 5, 1, 1, 2, 1, 4, 5, 2, 4, 1]
cost_per_km = 1

In [48]:
import plotly.express as px
import plotly.graph_objects as go

# Plot the problem configuration (facilities and customers)
fig = go.Figure()
fig.add_trace(go.Scatter(x=[x for x,y in facilities], y=[y for x,y in facilities], mode='markers', name='Facilities'))
fig.add_trace(go.Scatter(x=[x for x,y in customers], y=[y for x,y in customers], mode='markers', name='Hospital'))
fig.update_layout(title='Facilities and hospitals', xaxis_title='x', yaxis_title='y')
fig.show()

### Preprocesamiento
Definimos una función que determina la distancia euclidiana entre cada instalación y los sitios de los clientes. Además, calculamos los parámetros clave requeridos por la formulación del modelo MIP del problema de ubicación de la instalación.

In [49]:
# Determina la distancia euclidiana entre la bodega seleccionalada y los clientes

def compute_distance(loc1, loc2):
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return sqrt(dx*dx + dy*dy)


# computamos algunos parámetros auxiliares que se necesitán para construir el modelo MIP

num_facilities = len(facilities)
num_customers = len(customers)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))

# Computamos los costos de envío

shipping_cost = {(c,f): cost_per_km*compute_distance(customers[c], facilities[f]) for c, f in cartesian_prod}



## Despliegue del modelo

Ahora definimos el modelo MIP para el problema de ubicación de instalaciones, mediante la definición de las variables de decisión, restricciones y función objetivo. A continuación, iniciamos el proceso de optimización y Gurobi encuentra el plan para construir instalaciones que minimiza los costos totales.

In [50]:
import pulp as pl
from itertools import product

# MIP model formulation
m = pl.LpProblem('facility_location', pl.LpMinimize)

select = pl.LpVariable.dicts('Select', range(num_facilities), lowBound=0, upBound=1, cat='Binary')
assign = pl.LpVariable.dicts('Assign', cartesian_prod, lowBound=0, upBound=1, cat='Continuous')




In [51]:
for c,f in cartesian_prod:
     m += assign[(c,f)] <= select[f]
      
for c in range(num_customers):     
     m += pl.lpSum([assign[(c,f)] for f in range(num_facilities)]) == 1 

In [52]:
openings = pl.lpSum([select[f] for f in range(num_facilities)])
m+=openings 

In [53]:
m += pl.lpSum([select[f]*setup_cost[f] for f in range(num_facilities)]) + \
     pl.lpSum([assign[(c,f)]*shipping_cost[(c,f)] for (c,f) in cartesian_prod])
               
m.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/user/opt/anaconda3/envs/apricot-env/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/xc/t5gbnt3n5sq5bkg65xw9f1g00000gn/T/2131ff318d2f4ef8b46747b322168f66-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/xc/t5gbnt3n5sq5bkg65xw9f1g00000gn/T/2131ff318d2f4ef8b46747b322168f66-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 425 COLUMNS
At line 2086 RHS
At line 2507 BOUNDS
At line 2928 ENDATA
Problem MODEL has 420 rows, 420 columns and 1200 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 17.1994 - 0.00 seconds
Cgl0004I processed model has 420 rows, 420 columns (20 integer (20 of which binary)) and 1200 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 17.1994
Cbc0038I Relaxing continuous gives 17.1994


/Users/user/opt/anaconda3/envs/apricot-env/lib/python3.9/site-packages/pulp/pulp.py:1704: UserWarning:

Overwriting previously set objective.



1

## Análisis
El resultado del modelo de optimización muestra que el valor mínimo del costo total es de 4.72 millones de GBP. Veamos la solución que logra ese resultado óptimo.

## Plan de construcción de almacenes
Este plan determina en qué ubicaciones de sitios construir un almacén.

In [54]:
# display optimal values of decision variables using pulp notation

for k in select:
    if select[k].varValue > 0:
        print(select[k].name, '=', select[k].varValue)

for k in assign:
    if assign[k].varValue > 0:
        print(assign[k].name, '=', assign[k].varValue)

Select_0 = 1.0
Select_2 = 1.0
Select_9 = 1.0
Select_13 = 1.0
Select_14 = 1.0
Assign_(0,_0) = 1.0
Assign_(1,_13) = 1.0
Assign_(2,_9) = 1.0
Assign_(3,_9) = 1.0
Assign_(4,_14) = 1.0
Assign_(5,_0) = 1.0
Assign_(6,_13) = 1.0
Assign_(7,_9) = 1.0
Assign_(8,_9) = 1.0
Assign_(9,_14) = 1.0
Assign_(10,_0) = 1.0
Assign_(11,_13) = 1.0
Assign_(12,_13) = 1.0
Assign_(13,_2) = 1.0
Assign_(14,_14) = 1.0
Assign_(15,_0) = 1.0
Assign_(16,_13) = 1.0
Assign_(17,_13) = 1.0
Assign_(18,_2) = 1.0
Assign_(19,_0) = 1.0


In [55]:
# Depict the optimal solution in plotly figure, use lines to depict  allocation of customers to facilities

fig = go.Figure()
fig.add_trace(go.Scatter(x=[x for x,y in facilities], y=[y for x,y in facilities], mode='markers', name='Facilities'))
fig.add_trace(go.Scatter(x=[x for x,y in customers], y=[y for x,y in customers], mode='markers', name='Hospitals'))
for c,f in cartesian_prod:
    if assign[(c,f)].varValue > 0:
        fig.add_trace(go.Scatter(x=[customers[c][0], facilities[f][0]], y=[customers[c][1], facilities[f][1]], mode='lines', line=dict(color='gray', width=1), showlegend=False))
fig.update_layout(title='Facilities and hospitals', xaxis_title='x', yaxis_title='y')
fig.show()

### Plan de Envío
Este plan determina el porcentaje de envíos que se enviarán desde cada instalación construida a cada cliente.

In [56]:
# Shipments from facilities to customers.

for customer, facility in assign.keys():
    if (abs(assign[customer, facility].varValue) > 1e-6):
        print(f"\n Hospital {customer + 1} receives from Warehouse {facility + 1} {round(100*assign[customer, facility].varValue)}% of its needs")



 Hospital 1 receives from Warehouse 1 100% of its needs

 Hospital 2 receives from Warehouse 14 100% of its needs

 Hospital 3 receives from Warehouse 10 100% of its needs

 Hospital 4 receives from Warehouse 10 100% of its needs

 Hospital 5 receives from Warehouse 15 100% of its needs

 Hospital 6 receives from Warehouse 1 100% of its needs

 Hospital 7 receives from Warehouse 14 100% of its needs

 Hospital 8 receives from Warehouse 10 100% of its needs

 Hospital 9 receives from Warehouse 10 100% of its needs

 Hospital 10 receives from Warehouse 15 100% of its needs

 Hospital 11 receives from Warehouse 1 100% of its needs

 Hospital 12 receives from Warehouse 14 100% of its needs

 Hospital 13 receives from Warehouse 14 100% of its needs

 Hospital 14 receives from Warehouse 3 100% of its needs

 Hospital 15 receives from Warehouse 15 100% of its needs

 Hospital 16 receives from Warehouse 1 100% of its needs

 Hospital 17 receives from Warehouse 14 100% of its needs

 Hospital 

##  Conclusión
En este ejemplo, abordamos un problema de ubicación de instalaciones en el que deseamos construir almacenes para suministrar a una gran cantidad de hospitales, minimizando los costos totales fijos de construcción de almacenes y los costos variables totales de envío desde los almacenes hasta los hospitales. Aprendimos cómo formular el problema como un modelo MIP. También aprendimos cómo implementar la formulación del modelo MIP y resolverlo utilizando la API de Python de Cbc Solver. 

##  Referencias
[1] Laporte, Gilbert, Stefan Nickel, and Saldanha da Gama, Francisco. Location Science. Springer, 2015.